In [1]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [2]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

%load_ext autoreload
%autoreload 2

In [3]:
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

In [4]:
with open('../data/train_opt.pkl', mode='rb') as f:
    opt = pickle.load(f)
    
iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')

In [5]:
# opt.lr = 1e-6
opt.loadSize = 480
opt.label_nc = 0
opt.temp_data = True
opt.input_nc = 6
# opt.G_steps = 10
opt.no_flip = True
opt.model = 'pix2pixHD_tempSmooth'
opt.debug = True
opt.name = 'maskGirl'
opt.dataroot='../data/target/maskGirl/'
opt.continue_train = True
opt.load_pretrain = '../checkpoints/maskGirl/'

In [6]:
if opt.continue_train:
    try:
        start_epoch, epoch_iter = np.loadtxt(iter_path , delimiter=',', dtype=int)
    except:
        start_epoch, epoch_iter = 1, 0
    print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))        
else:    
    start_epoch, epoch_iter = 1, 0

if opt.debug:
    opt.display_freq = 1
    opt.print_freq = 1
    opt.niter = 40
    opt.niter_decay = 40
    opt.max_dataset_size = 10
print(opt)

Resuming from epoch 21 at iteration 0
Namespace(batchSize=1, beta1=0.5, checkpoints_dir='../checkpoints/', continue_train=True, data_type=32, dataroot='../data/target/maskGirl/', debug=True, display_freq=1, display_winsize=512, feat_num=3, fineSize=512, fine_size=480, gpu_ids=[0], input_nc=6, instance_feat=False, isTrain=True, label_feat=False, label_nc=0, lambda_feat=10.0, loadSize=480, load_features=False, load_pretrain='../checkpoints/maskGirl/', lr=0.0002, max_dataset_size=10, model='pix2pixHD_tempSmooth', nThreads=2, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_layers_D=3, n_local_enhancers=1, name='maskGirl', ndf=64, nef=16, netG='global', ngf=64, niter=40, niter_decay=40, niter_fix_global=0, no_flip=True, no_ganFeat_loss=False, no_html=False, no_instance=True, no_lsgan=False, no_vgg_loss=False, norm='instance', num_D=2, output_nc=3, phase='train', pool_size=0, print_freq=1, resize_or_crop='scale_width', save_epoch_freq=10, save_l

In [7]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
    
# start_epoch, epoch_iter = 1, 0
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

CustomDatasetDataLoader
dataset [AlignedDataset] was created
#training images = 10


In [8]:
model = create_model(opt)
visualizer = Visualizer(opt)

GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(6, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_runn

create web directory ../checkpoints/maskGirl/web...


In [9]:
step = 0
for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
    for i, data in enumerate(dataset, start=epoch_iter):
        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        save_fake = total_steps % opt.display_freq == display_delta
        
        ############## Forward Pass ######################
        losses, generated = model(Variable(data['label']), Variable(data['inst']), 
            Variable(data['image']), Variable(data['feat']), infer=save_fake)
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5
        loss_G = loss_dict['G_GAN'] + loss_dict.get('G_GAN_Feat',0) + loss_dict.get('G_VGG',0)
        
        ############### Backward Pass ####################
        # update generator weights
        model.module.optimizer_G.zero_grad()
        loss_G.backward()
        model.module.optimizer_G.step()
        model.module.optimizer_D.zero_grad()
#         step += 1

        # update discriminator weights
        model.module.optimizer_D.zero_grad()
        loss_D.backward()
        model.module.optimizer_D.step()
#         step = 0
        #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]) 

        ############## Display results and errors ##########
        ### print out errors
        if total_steps % opt.print_freq == print_delta:
            errors = {k: v.data[0] if not isinstance(v, int) else v for k, v in loss_dict.items()}
            t = (time.time() - iter_start_time) / opt.batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            visualizer.plot_current_errors(errors, total_steps)

        ### display output images
        if save_fake:
            visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0][3:,:,:], opt.label_nc)),
                                   ('synthesized_image', util.tensor2im(generated.data[0])),
                                   ('real_image', util.tensor2im(data['image'][0][3:,:,:]))])
            visualizer.display_current_results(visuals, epoch, total_steps)

        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.module.save('latest')            
            np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

        if epoch_iter >= dataset_size:
            break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()
        
torch.cuda.empty_cache()

/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


(epoch: 21, iters: 1, time: 4.748) G_GAN: 2.481 G_GAN_Feat: 9.891 G_VGG: 12.987 D_real: 0.188 D_fake: 0.195 
(epoch: 21, iters: 2, time: 0.519) G_GAN: 1.823 G_GAN_Feat: 9.518 G_VGG: 12.894 D_real: 2.270 D_fake: 3.130 
(epoch: 21, iters: 3, time: 0.528) G_GAN: 2.358 G_GAN_Feat: 9.456 G_VGG: 13.179 D_real: 1.698 D_fake: 2.109 
(epoch: 21, iters: 4, time: 0.565) G_GAN: 3.140 G_GAN_Feat: 8.244 G_VGG: 12.897 D_real: 1.568 D_fake: 0.915 
(epoch: 21, iters: 5, time: 0.552) G_GAN: 2.469 G_GAN_Feat: 7.838 G_VGG: 12.311 D_real: 0.826 D_fake: 0.492 
(epoch: 21, iters: 6, time: 0.607) G_GAN: 1.408 G_GAN_Feat: 8.014 G_VGG: 12.105 D_real: 0.302 D_fake: 0.285 
(epoch: 21, iters: 7, time: 0.541) G_GAN: 1.131 G_GAN_Feat: 8.127 G_VGG: 12.260 D_real: 0.241 D_fake: 0.517 
(epoch: 21, iters: 8, time: 0.544) G_GAN: 1.368 G_GAN_Feat: 7.447 G_VGG: 11.612 D_real: 0.240 D_fake: 0.302 
(epoch: 21, iters: 9, time: 0.518) G_GAN: 1.430 G_GAN_Feat: 6.717 G_VGG: 11.339 D_real: 0.255 D_fake: 0.284 
(epoch: 21, iters: 

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f52d06fc6a0>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 58, in detach
    return reduction.recv_handle(conn)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/pytho

(epoch: 22, iters: 1, time: 0.540) G_GAN: 1.583 G_GAN_Feat: 6.384 G_VGG: 11.184 D_real: 0.398 D_fake: 0.272 
(epoch: 22, iters: 2, time: 0.549) G_GAN: 1.179 G_GAN_Feat: 6.558 G_VGG: 11.040 D_real: 0.379 D_fake: 0.492 
(epoch: 22, iters: 3, time: 0.560) G_GAN: 1.406 G_GAN_Feat: 6.176 G_VGG: 10.974 D_real: 0.288 D_fake: 0.402 
(epoch: 22, iters: 4, time: 0.536) G_GAN: 1.936 G_GAN_Feat: 7.190 G_VGG: 10.886 D_real: 0.357 D_fake: 0.197 
(epoch: 22, iters: 5, time: 0.514) G_GAN: 1.153 G_GAN_Feat: 6.687 G_VGG: 10.466 D_real: 0.494 D_fake: 0.466 
(epoch: 22, iters: 6, time: 0.533) G_GAN: 1.319 G_GAN_Feat: 7.415 G_VGG: 10.942 D_real: 0.314 D_fake: 0.405 
(epoch: 22, iters: 7, time: 0.535) G_GAN: 1.044 G_GAN_Feat: 6.362 G_VGG: 10.819 D_real: 0.276 D_fake: 0.502 
(epoch: 22, iters: 8, time: 0.521) G_GAN: 2.448 G_GAN_Feat: 6.801 G_VGG: 10.360 D_real: 0.653 D_fake: 0.336 
(epoch: 22, iters: 9, time: 0.534) G_GAN: 1.273 G_GAN_Feat: 6.013 G_VGG: 10.155 D_real: 0.517 D_fake: 0.582 
(epoch: 22, iters: 

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f52d07a5160>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/yaosy/anaconda3/e

...

Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f52c6c489e8>>
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/home/yaosy/anaconda3/e

(epoch: 74, iters: 1, time: 0.535) G_GAN: 1.241 G_GAN_Feat: 5.757 G_VGG: 7.911 D_real: 0.100 D_fake: 0.183 
(epoch: 74, iters: 2, time: 0.529) G_GAN: 2.089 G_GAN_Feat: 6.109 G_VGG: 7.416 D_real: 0.030 D_fake: 0.016 
(epoch: 74, iters: 3, time: 0.522) G_GAN: 1.946 G_GAN_Feat: 7.128 G_VGG: 8.498 D_real: 0.042 D_fake: 0.021 
(epoch: 74, iters: 4, time: 0.521) G_GAN: 2.009 G_GAN_Feat: 6.065 G_VGG: 8.178 D_real: 0.049 D_fake: 0.019 
(epoch: 74, iters: 5, time: 0.525) G_GAN: 2.089 G_GAN_Feat: 5.602 G_VGG: 7.266 D_real: 0.022 D_fake: 0.013 
(epoch: 74, iters: 6, time: 0.507) G_GAN: 1.789 G_GAN_Feat: 6.599 G_VGG: 8.412 D_real: 0.029 D_fake: 0.026 
(epoch: 74, iters: 7, time: 0.532) G_GAN: 1.896 G_GAN_Feat: 5.628 G_VGG: 7.676 D_real: 0.015 D_fake: 0.016 
(epoch: 74, iters: 8, time: 0.535) G_GAN: 2.127 G_GAN_Feat: 6.260 G_VGG: 8.174 D_real: 0.025 D_fake: 0.019 
(epoch: 74, iters: 9, time: 0.525) G_GAN: 2.050 G_GAN_Feat: 6.048 G_VGG: 7.856 D_real: 0.011 D_fake: 0.014 
(epoch: 74, iters: 10, time:

Process Process-115:
Process Process-116:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/ya

(epoch: 78, iters: 2, time: 0.536) G_GAN: 1.566 G_GAN_Feat: 5.275 G_VGG: 7.099 D_real: 0.016 D_fake: 0.042 


KeyboardInterrupt: 

In [ ]:
plt.imshow(util.tensor2im(model.module.Gt1.squeeze(0)))

In [ ]:
plt.imshow(util.tensor2label(data['label'][0][:3,:,:], opt.label_nc))

In [ ]:
plt.imshow(util.tensor2label(data['label'][0][3:,:,:], opt.label_nc))

In [ ]:
plt.imshow(util.tensor2im(data['image'][0][3:,:,:]))

In [ ]:
plt.imshow(util.tensor2im(data['image'][0][:3,:,:]))